# Code snippets:

In [9]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [10]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

NotFoundError: Requested device not found


# Code:

In [3]:
%%capture
!pip install timm 

In [1]:
# Import depencies:
import cv2
import torch
import matplotlib.pyplot as plt

In [5]:
%%capture
# Download the MiDaS
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
midas.to('cpu')
midas.eval()

In [ ]:
%%capture
# Input transformational pipeline
transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transforms = transforms.small_transform 

In [7]:
# Hook into OpenCV
cap = cv2.VideoCapture(0)
while cap.isOpened():
  ret, frame = cap.read()

  # Transform input for MiDaS:
  img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  imgbatch = transform(img).to('cpu')

  # Make a prediction
  with torch.no_grad():
    prediction = midas(imgbatch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size = img.shape[:2],
        mode = 'bicubic'
        align_corners=False
    ) .squeeze()

    output = prediction.cpu().numpy()

    print(prediction)

  plt.imshow(output)
  cv2.imshow('CV2Frame', frame)
  plt.pause(0.00001)

  if cv2.waitKey(10) & OxFF == ord('q'):
    cap.release()
    cv2.destroyAllWindows()

plt.show()